In [1]:
pip install pandas_market_calendars

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.9/123.9 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 14.4 MB/s eta 0:00:00


In [3]:
import yfinance as yf
import pandas as pd
import os
from datetime import datetime, timedelta
import pandas_market_calendars as mcal
import os

data_path = os.path.join("..", "data", "pred_data.csv")

# Define the ticker symbol for Nifty 50
nifty50_ticker = "^NSEI"

# Use today's date as the end date
end_date = datetime.today() +timedelta(days=1)

# Calculate the start date (100 days before the end date)
start_date = end_date - timedelta(days=100)  # Giving enough buffer to find 30 trading days

# Convert start and end dates to string format
start_date_str = start_date.strftime('%Y-%m-%d')
end_date_str = end_date.strftime('%Y-%m-%d')

# Initialize the calendar for the Indian stock market (NSE)
nse = mcal.get_calendar('NSE')

# Get the list of all trading days between the start and end date
valid_trading_days = nse.valid_days(start_date=start_date, end_date=end_date)

# Ensure we are considering exactly 60 trading days
valid_trading_days = valid_trading_days[-67:]  # Get the last 60 trading days

# Fetch data for the 30 trading days
nifty50_data = yf.download(nifty50_ticker, start=valid_trading_days[0].strftime('%Y-%m-%d'),
                           end=valid_trading_days[-1].strftime('%Y-%m-%d'), interval="1d")

# Check if the data is fetched successfully
if nifty50_data.empty:
    print(f"Data for {nifty50_ticker} is unavailable. Please verify the ticker symbol.")
else:
    # Reset index to make the date a column
    nifty50_data.reset_index(inplace=True)

    # Ensure the date format is YYYY-MM-DD
    nifty50_data['Date'] = pd.to_datetime(nifty50_data['Date']).dt.strftime('%Y-%m-%d')

    # Check if the file already exists
    file_name = "nifty50_30_days_data.csv"
    if os.path.exists(file_name):
        # Load existing data
        existing_data = pd.read_csv(file_name, index_col=0)

        # Combine the existing data with the new data, avoiding duplicates
        updated_data = pd.concat([existing_data, nifty50_data[~nifty50_data['Date'].isin(existing_data['Date'])]])
    else:
        # If the file does not exist, create it with the fetched data
        updated_data = nifty50_data

    # Save the updated data back to the file
    updated_data.to_csv("data_path", index=False)

    # Print the first few rows to verify
    print(updated_data.head())


import pandas as pd

# Load the dataset into a pandas DataFrame
data = pd.read_csv("data_path")

# Remove the first row (index 0)
data = data.drop(index=0)

# Reset the index if you want to have a clean index after dropping the row
data = data.reset_index(drop=True)

# Optionally, save the modified DataFrame to a new CSV file
data.to_csv("data_path", index=False)
print("---------------------------------------------------------------------------")
print(data.tail())


[*********************100%***********************]  1 of 1 completed

Price         Date         Close          High           Low          Open  \
Ticker                     ^NSEI         ^NSEI         ^NSEI         ^NSEI   
0       2025-03-04  22082.650391  22105.050781  21964.599609  21974.449219   
1       2025-03-05  22337.300781  22394.900391  22067.800781  22073.050781   
2       2025-03-06  22544.699219  22556.449219  22245.849609  22476.349609   
3       2025-03-07  22552.500000  22633.800781  22464.750000  22508.650391   
4       2025-03-10  22460.300781  22676.750000  22429.050781  22521.849609   

Price   Volume  
Ticker   ^NSEI  
0       253300  
1       305900  
2       372100  
3       289800  
4       293900  
---------------------------------------------------------------------------
          Date            Close             High              Low  \
55  2025-05-28   24752.44921875         24864.25   24737.05078125   
56  2025-05-29  24833.599609375  24892.599609375   24677.30078125   
57  2025-05-30   24750.69921875   24863.94921875  2